In [1]:
import geopandas as gpd
import pandas as pd
import altair as alt
from pathlib import Path

ROOT = Path("..")
TRAFFIC_DATA = ROOT / "data" / "traffic"

In [2]:
counts = pd.read_csv(TRAFFIC_DATA / "rawcount.csv", index_col=["count_point_id", "count_date"], parse_dates=["count_date"])

In [3]:
count_points = pd.read_csv(TRAFFIC_DATA / "countpoints.csv", index_col=["count_point_id"])

In [4]:
count_points

,region_id,region_name,local_authority_id,local_authority_name,road_name,road_type,start_junction_road_name,end_junction_road_name,easting,northing,latitude,longitude,link_length_km,link_length_miles
count_point_id,,,,,,,,,,,,,,
990295,1,South West,115,Bath and North East Somerset,C,Minor,NaN,NaN,357650,161220,51.348473,-2.609475,NaN,NaN
990073,1,South West,115,Bath and North East Somerset,B3115,Minor,NaN,NaN,365480,159140,51.330302,-2.496850,NaN,NaN
990072,1,South West,115,Bath and North East Somerset,B3111,Minor,NaN,NaN,374270,164020,51.374648,-2.371045,NaN,NaN
966018,1,South West,115,Bath and North East Somerset,C,Minor,NaN,NaN,373543,164166,51.375928,-2.381501,NaN,NaN
964896,1,South West,115,Bath and North East Somerset,U,Minor,NaN,NaN,378100,168000,51.410596,-2.316268,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7128,1,South West,115,Bath and North East Somerset,A367,Major,Avon St,A4,374700,164540,51.379343,-2.364905,0.7,0.43
6990,1,South West,115,Bath and North East Somerset,A36,Major,A367 roundabout,A3062 roundabout,375300,164240,51.376671,-2.356263,0.5,0.31
6382,1,South West,115,Bath and North East Somerset,A37,Major,B3130,Staunton Lane,361760,165340,51.385808,-2.550909,3.2,1.99


In [5]:
WIDTH = 1000
HEIGHT = 1000

In [6]:
count_points_plot = (
    alt.Chart(count_points)
    .mark_point()
    .encode(
        latitude="latitude:Q",
        longitude="longitude:Q",
        tooltip=["road_name", "latitude", "longitude", "start_junction_road_name", "end_junction_road_name"]
    )
    .project("mercator")
    .properties(width=WIDTH, height=HEIGHT)
)

In [7]:
banes_boundary = gpd.read_file(TRAFFIC_DATA / "os_boundary.geojson")
boundary_plot = alt.Chart(banes_boundary).mark_geoshape(fill="lightgray").project("mercator").properties(width=WIDTH, height=HEIGHT)

In [8]:
banes_roads = gpd.read_file(TRAFFIC_DATA / "os_roads.geojson")
banes_roads["lat"] = banes_roads.geometry.apply(lambda g: g.centroid.y)  # no mistake
banes_roads["lon"] = banes_roads.geometry.apply(lambda g: g.centroid.x)
banes_roads.head()

,id,distname,roadnumber,classifica,drawlevel,override,featcode,geometry,lat,lon
0,88100875-34A5-40A2-870D-1F46774B4C52,Margaret's Hill,None,Local Road,0,F,15760,"LINESTRING (-2.35845 51.39020, -2.35846 51.390...",51.390433,-2.358457
1,2E3B0DB9-9752-41B6-9C01-052F22C56B84,Pera Road,None,Local Road,0,F,15760,"LINESTRING (-2.35845 51.39020, -2.35822 51.390...",51.390442,-2.357686
2,A40838A7-DFBE-4525-BBA9-881F5F6D7E7B,Lansdown Grove,None,Local Road,0,F,15760,"LINESTRING (-2.36214 51.39092, -2.36257 51.390...",51.390564,-2.363191
3,3AB5B394-DE10-4844-AE13-2A1267ACDED0,Lansdown Road,None,Minor Road,0,F,15750,"LINESTRING (-2.36421 51.39098, -2.36433 51.390...",51.390633,-2.364281
4,6372A3E2-D2EC-4CE4-B085-9251D20F0C23,Alpine Gardens,None,Local Road,0,F,15760,"LINESTRING (-2.35911 51.39108, -2.35922 51.390...",51.390778,-2.359225


In [9]:
banes_roads_plot = (
    alt.Chart(banes_roads.dropna(subset=["roadnumber"]))
    .mark_geoshape(
        fill=None,
        stroke="orange"
    )
    .project("mercator")
    .properties(width=WIDTH, height=HEIGHT)
)

In [10]:
banes_roads_labels = (
    alt.Chart(banes_roads.dropna(subset=["roadnumber", "distname"]))
    .mark_text()
    .encode(
        latitude="lat",
        longitude="lon",
        text="distname"
    )
    .project("mercator")
    .properties(width=WIDTH, height=HEIGHT)
)

In [11]:
count_points_labels = (
    alt.Chart(count_points.query("road_type == 'Major'"))
    .mark_text()
    .encode(
        latitude="latitude:Q",
        longitude="longitude:Q",
        text="road_name"
    )
    .project("mercator")
    .properties(width=WIDTH, height=HEIGHT)
)

In [12]:
wards = gpd.read_file(TRAFFIC_DATA / "banes_boundary_review_2018_final_wards_simp.geojson")

In [13]:
wards["lat"] = wards.geometry.apply(lambda g: g.centroid.y)  # no mistake
wards["lon"] = wards.geometry.apply(lambda g: g.centroid.x)

In [14]:
wards_plot = alt.Chart(wards).mark_geoshape(
    fill="lightgray",
    stroke="gray"
).properties(width=WIDTH, height=HEIGHT)

In [15]:
wards_labels = (
    alt.Chart(wards)
    .mark_text()
    .encode(
        latitude="lat:Q",
        longitude="lon:Q",
        text="Name"
    )
    .properties(width=WIDTH, height=HEIGHT)
)

In [16]:
(
    wards_plot 
    + wards_labels 
    + banes_roads_plot 
#    + banes_roads_labels 
    + count_points_plot 
    + count_points_labels
)

alt.LayerChart(...)